## **Ingesta del archivo `results.json`**

- Se incorpora una variable de parámetro en la ruta **.json(f"{raw_folder_path}/`{v_file_date}`/results.json")** al momento de leer el archivo.
- De este modo, especificamos la fecha desde el notebook padre, lo que permite cargar únicamente aquellos archivos ubicados en un directorio que coincida con dicha fecha.

In [ ]:
# Creamos un parametro para el nombre del archivo
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
# Creamos un parametro para la fecha del archivo
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
# Llamamos al notebook que contiene las variables de configuración
%run "../../utils/configuration"

In [ ]:
# Llamamos al notebook que contiene funciones comunes
%run "../../utils/common_functions"

### Paso 1 - Leer el archivo JSON

In [ ]:
from pyspark.sql.functions import col, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [ ]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("grid", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("positionText", StringType(), True),
                                    StructField("positionOrder", IntegerType(), True),
                                    StructField("points", FloatType(), True),
                                    StructField("laps", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True),
                                    StructField("fastestLap", IntegerType(), True),
                                    StructField("rank", IntegerType(), True),
                                    StructField("fastestLapTime", StringType(), True),
                                    StructField("fastestLapSpeed", FloatType(), True),
                                    StructField("statusId", StringType(), True)])

In [ ]:
# El parámetro "raw_folder_path" se encuentra en el notebook "configuration"
# El parámetro "v_file_date" se encuentra en el notebook e indicamos su valor en tiempo de ejecución
# Ejemplo para la fecha 2021-03-21: esto seria equivalente a la ruta: /mnt/formula1dl/raw/2021-03-21/results.json
results_df = spark.read \
.schema(results_schema) \
.json(f"{raw_folder_path}/{v_file_date}/results.json")

### Paso 2 - Renombrar columnas y añadir columnas nuevas

In [ ]:
results_with_columns_df = results_df.withColumnRenamed("resultId", "result_id") \
                                    .withColumnRenamed("raceId", "race_id") \
                                    .withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("constructorId", "constructor_id") \
                                    .withColumnRenamed("positionText", "position_text") \
                                    .withColumnRenamed("positionOrder", "position_order") \
                                    .withColumnRenamed("fastestLap", "fastest_lap") \
                                    .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                                    .withColumn("data_source", lit(v_data_source)) \
                                    .withColumn("file_date", lit(v_file_date))

In [ ]:
# La función "add_ingestion_date()" se encuentra en el notebook "common_functions"
# La fecha de ingestión será del tipo timestamp
results_with_ingestion_date_df = add_ingestion_date(results_with_columns_df)

### Paso 3 - Eliminar la columna no deseada

In [ ]:
results_final_df = results_with_ingestion_date_df.drop(col("statusId"))

### Paso 4 - Escribir datos en el datalake como parquet y crear la tabla **results** en la base de datos **f1_processed**

In [ ]:
# La función "overwrite_partition()" se encuentra en el notebook "common_functions"
overwrite_partition(results_final_df, 'f1_processed', 'results', 'race_id')

In [ ]:
dbutils.notebook.exit("Success")